In [ ]:
# ============================================================
# Setup (run once per fresh Colab runtime)
# ============================================================
!pip -q install \
  rasterio==1.3.8 geopandas==0.14.0 shap==0.45.0 xgboost==2.0.3 \
  scikit-learn==1.5.0 matplotlib==3.8.4 pandas==2.2.2 tqdm==4.66.4 \
  scipy==1.11.4 pyogrio==0.9.0 pyproj==3.6.1 rtree==1.2.0 \
  torch==2.3.0 torchvision==0.18.0 -f https://download.pytorch.org/whl/cpu/torch_stable.html

# ============================================================
# Imports
# ============================================================
import os, json, numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt
import rasterio, rasterio.mask
from rasterio.enums import Resampling
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier
import shap
from scipy.spatial import cKDTree
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ============================================================
# A) Suitability — sample generator (with extra negatives)
# ============================================================
raster_files = {
    "elevation": "elevation.tif",
    "slope": "slope.tif",
    "aspect": "aspect.tif",
    "runoff": "runoff.tif",
    "distance_water": "distance_water.tif",
    "distance_road": "distance_road.tif",
    "distance_rail": "distance_rail.tif",
    "wetland": "wetland.tif",
    "pop_density": "pop_density.tif",
    "ecoreserve": "ecoreserve.tif"
}
rasters = {k: rasterio.open(v) for k, v in raster_files.items()}
ref_raster = rasters["slope"]; W, H = ref_raster.width, ref_raster.height; transform = ref_raster.transform

def score_pixel(vals):
    score = 0
    def safe(v): return v if -1e10 < v < 1e10 else np.nan
    if safe(vals["distance_water"]) < 500: score += 1
    if safe(vals["slope"]) < 5: score += 1
    if safe(vals["runoff"]) > 4: score += 1
    if safe(vals["pop_density"]) > 25: score += 1
    if safe(vals["distance_road"]) < 500: score += 1
    if safe(vals["distance_rail"]) < 1000: score += 1
    if safe(vals["distance_water"]) > 2000: score -= 1
    if safe(vals["slope"]) > 15: score -= 1
    if safe(vals["runoff"]) < 1: score -= 1
    if safe(vals["pop_density"]) < 5: score -= 1
    if safe(vals["distance_road"]) > 3000: score -= 1
    if vals["ecoreserve"] == 1: score -= 1
    return score

np.random.seed(42); target_per_class = 200; max_attempts = 100000
class_1, class_0, neutral = [], [], []
with tqdm(total=max_attempts) as pbar:
    while len(class_1)<target_per_class or len(class_0)<target_per_class or len(neutral)<target_per_class:
        row, col = np.random.randint(0,H), np.random.randint(0,W)
        x,y = rasterio.transform.xy(transform, row, col)
        vals, valid = {}, True
        for name, r in rasters.items():
            try:
                v = list(r.sample([(x,y)]))[0][0]
                if v in [-9999,-99999] or np.isnan(v): valid=False; break
                vals[name]=v
            except: valid=False; break
        if not valid: pbar.update(1); continue
        s = score_pixel(vals)
        if s>=3 and len(class_1)<target_per_class: class_1.append((x,y,1,*vals.values()))
        elif s<=-1 and len(class_0)<target_per_class: class_0.append((x,y,0,*vals.values()))
        elif -1<s<3 and len(neutral)<target_per_class: neutral.append((x,y,0,*vals.values()))
        pbar.update(1);
        if pbar.n>=max_attempts: break

final_class_0 = class_0 + neutral[:(target_per_class-len(class_0))]
final_data = class_1 + final_class_0
df_samples = pd.DataFrame(final_data, columns=["x","y","label"]+list(raster_files.keys()))
df_samples.to_csv("samples.csv", index=False)
print("✅ samples.csv saved:", df_samples.shape)

# ============================================================
# B) XGBoost + SHAP on points (baseline LR/RF comparison)
# ============================================================
samples = pd.read_csv("samples.csv")
drivers = list(raster_files.keys())
def extract_raster_values(sample_df, tif_list):
    feats=[]
    for f in tif_list:
        with rasterio.open(f) as src:
            vals = [list(src.sample([(x,y)]))[0][0] for x,y in zip(sample_df.x, sample_df.y)]
            feats.append(vals)
    arr = np.array(feats).T
    return pd.DataFrame(arr, columns=[os.path.splitext(os.path.basename(f))[0] for f in tif_list])

X = extract_raster_values(samples, [f"{k}.tif" for k in drivers])
y = samples["label"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

xgb_points = XGBClassifier()
xgb_points.fit(X_train,y_train)
print("Train acc:", xgb_points.score(X_train,y_train), " Test acc:", xgb_points.score(X_test,y_test))

# Baselines
def eval_model(model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs, prs = [], []
    for tr,te in cv.split(X,y):
        model.fit(X.iloc[tr], y.iloc[tr])
        p = model.predict_proba(X.iloc[te])[:,1]
        aucs.append(roc_auc_score(y.iloc[te], p))
        prs.append(average_precision_score(y.iloc[te], p))
    return np.mean(aucs), np.mean(prs)

lr_auc, lr_pr = eval_model(LogisticRegression(max_iter=200, class_weight="balanced", solver="liblinear"), X, y)
rf_auc, rf_pr = eval_model(RandomForestClassifier(n_estimators=500,n_jobs=-1,class_weight="balanced",random_state=42), X, y)
xgb_auc, xgb_pr = eval_model(XGBClassifier(), X, y)
pd.DataFrame({"model":["XGBoost","Logistic","RandomForest"],"AUC":[xgb_auc,lr_auc,rf_auc],"PR-AUC":[xgb_pr,lr_pr,rf_pr]}).to_csv("/content/baseline_comparison.csv", index=False)

# SHAP
expl = shap.Explainer(xgb_points); sv = expl(X)
shap.summary_plot(sv, X, show=False); plt.show()
shap.plots.bar(sv, max_display=10, show=False); plt.show()

# ============================================================
# C) Align rasters to reference slope.tif -> /content/aligned
# ============================================================
input_dir = "/content"; aligned_dir = "/content/aligned"; os.makedirs(aligned_dir, exist_ok=True)
ref_file = os.path.join(input_dir, "slope.tif")
with rasterio.open(ref_file) as ref:
    ref_profile, ref_transform, ref_crs = ref.profile, ref.transform, ref.crs
    ref_shape = (ref.height, ref.width)

for fname in os.listdir(input_dir):
    if not fname.endswith(".tif"): continue
    src_path = os.path.join(input_dir,fname); dst_path=os.path.join(aligned_dir,fname)
    with rasterio.open(src_path) as src:
        data = src.read(out_shape=(src.count, ref_shape[0], ref_shape[1]), resampling=Resampling.bilinear)
        profile = src.profile
        profile.update(height=ref_shape[0], width=ref_shape[1], transform=ref_transform, crs=ref_crs)
        with rasterio.open(dst_path,"w",**profile) as dst: dst.write(data)
print("✅ Aligned → /content/aligned")

# ============================================================
# D) Suitability mapping on aligned rasters + IDW fill + 4 outputs
# ============================================================
raster_root = aligned_dir
study_shp = "/content/study_area.shp"
raster_files_map = {
    "elevation":"elevation.tif","slope":"slope.tif","aspect":"aspect.tif",
    "runoff":"runoff.tif","distance_water":"distance_water.tif",
    "distance_road":"distance_road.tif","distance_rail":"distance_rail.tif",
    "wetland":"wetland.tif","pop_density":"pop_density.tif","ecoreserve":"ecoreserve.tif"
}
def score_balanced(vals):
    score=0
    def safe(v): return v if -1e10 < v < 1e10 else np.nan
    if safe(vals["distance_water"])<400: score+=1
    if safe(vals["slope"])<6: score+=1
    if safe(vals["runoff"])>4: score+=1
    if safe(vals["pop_density"])>20: score+=1
    if safe(vals["distance_road"])<700: score+=1
    if safe(vals["distance_rail"])<1200: score+=1
    if safe(vals["distance_water"])>2500: score-=1
    if safe(vals["slope"])>12: score-=1
    if safe(vals["runoff"])<1: score-=1
    if safe(vals["pop_density"])<8: score-=1
    if safe(vals["distance_road"])>3500: score-=1
    if vals["ecoreserve"]==1: score-=1
    return score

# sample for training
ras_map = {k:rasterio.open(os.path.join(raster_root,v)) for k,v in raster_files_map.items()}
ref = ras_map["slope"]; W,H = ref.width, ref.height; transform = ref.transform
np.random.seed(42); pos,neg=[],[]
with tqdm(total=200000) as pbar:
    while len(pos)<300 or len(neg)<300:
        row, col = np.random.randint(0,H), np.random.randint(0,W)
        x,y = rasterio.transform.xy(transform,row,col)
        vals, valid = {}, True
        for name,r in ras_map.items():
            try:
                v = list(r.sample([(x,y)]))[0][0]
                if v in [-9999,-99999] or np.isnan(v): valid=False; break
                vals[name]=v
            except: valid=False; break
        if not valid: pbar.update(1); continue
        s = score_balanced(vals)
        if s>=4 and len(pos)<300: pos.append((x,y,1,*vals.values()))
        elif s<=1 and len(neg)<300: neg.append((x,y,0,*vals.values()))
        pbar.update(1)
        if pbar.n>=200000: break

cols = list(raster_files_map.keys())
df_map = pd.DataFrame(pos+neg, columns=["x","y","label"]+cols)
X_map, y_map = df_map[cols], df_map["label"]
xgb_map = XGBClassifier(n_estimators=100,max_depth=4,learning_rate=0.05,subsample=0.8,colsample_bytree=0.8,eval_metric="logloss").fit(X_map,y_map)

# predict full map
stacked=[]; profile=None
for f in raster_files_map.values():
    with rasterio.open(os.path.join(raster_root,f)) as src:
        stacked.append(src.read(1))
        if profile is None: profile=src.profile
stacked = np.stack(stacked,axis=-1)
flat = stacked.reshape(-1, len(cols))
valid_mask = ~np.any(np.isnan(flat),axis=1)
pred = np.full(flat.shape[0], np.nan)
pred[valid_mask] = xgb_map.predict_proba(flat[valid_mask])[:,1]
suitability_full = pred.reshape((H,W))

# mask to AOI
gdf = gpd.read_file(study_shp)
with rasterio.open(os.path.join(raster_root,"slope.tif")) as src:
    out_img,_ = rasterio.mask.mask(src, gdf.geometry, crop=False)
    study_mask = out_img[0] != src.nodata
suitability_masked = np.where(study_mask, suitability_full, np.nan)

# IDW fill
def idw_fill(raster, mask, power=2, k=8):
    filled = raster.copy()
    rows, cols = np.where(~np.isnan(raster))
    if len(rows)==0: return raster
    known_coords = np.c_[rows,cols]; known_vals = raster[rows,cols]
    t_rows, t_cols = np.where(mask)
    if len(t_rows)==0: return raster
    target_coords = np.c_[t_rows,t_cols]
    tree = cKDTree(known_coords)
    dists, idxs = tree.query(target_coords, k=k)
    weights = 1 / (dists**power + 1e-12); weights /= weights.sum(axis=1, keepdims=True)
    interp = (weights * known_vals[idxs]).sum(axis=1)
    filled[t_rows, t_cols] = interp
    return filled

idw_mask = np.isnan(suitability_masked) & study_mask
suitability_filled = idw_fill(suitability_masked, idw_mask)

def save_tif(arr, path, profile):
    profile.update(dtype=rasterio.float32, count=1)
    with rasterio.open(path,"w",**profile) as dst: dst.write(arr.astype(np.float32),1)

full_map  = suitability_filled
high_map  = np.where(full_map>=0.7, full_map, np.nan)
mid_map   = np.where((full_map>=0.3)&(full_map<0.7), full_map, np.nan)
low_map   = np.where((full_map>0)&(full_map<0.3), full_map, np.nan)

save_tif(full_map, "/content/ua_suitability_full.tif", profile)
save_tif(high_map, "/content/ua_suitability_high.tif", profile)
save_tif(mid_map,  "/content/ua_suitability_medium.tif", profile)
save_tif(low_map,  "/content/ua_suitability_low.tif", profile)

plt.figure(figsize=(8,5)); plt.imshow(full_map,cmap="YlGn",vmin=0,vmax=1)
plt.title("Urban Agriculture Suitability Map (IDW-Filled)"); plt.axis("off"); plt.colorbar(); plt.tight_layout(); plt.show()

# ============================================================
# E) Unconstrained scenario (MLP): 2010→2017 training → simulate 2030 & 2050
# ============================================================
rroot = aligned_dir
landuse_2010 = f"{rroot}/2010_9.tif"
landuse_2017 = f"{rroot}/2017_9.tif"
study_shp = "/content/study_area.shp"
suitability_map = f"{rroot}/suitability.tif"  # optional

def read_r(path):
    with rasterio.open(path) as src: return src.read(1), src.profile

def list_drivers():
    exclude = {"2010_9.tif","2017_9.tif","simulated_landuse_2030.tif","simulated_landuse_2050.tif","suitability.tif"}
    return [f for f in os.listdir(rroot) if f.endswith(".tif") and f not in exclude]

def load_mask(shp_path, ref_raster_path):
    gdf = gpd.read_file(shp_path)
    with rasterio.open(ref_raster_path) as src:
        out, _ = rasterio.mask.mask(src, gdf.geometry, crop=False, filled=False)
        return out[0].astype(bool)

def prepare_X_y(landuse_t1, landuse_t2):
    mask = load_mask(study_shp, landuse_t1)
    y1,_ = read_r(landuse_t1); y2,_ = read_r(landuse_t2)
    drivers=[]
    for fname in sorted(list_drivers()):
        d,_ = read_r(os.path.join(rroot,fname)); drivers.append(d)
    if os.path.exists(suitability_map):
        s,_ = read_r(suitability_map); drivers.append(s)
    X = np.stack(drivers,axis=-1)
    valid = np.all(~np.isnan(X),axis=-1) & ~np.isnan(y1) & ~np.isnan(y2) & mask
    X = X[valid]; y = y2[valid].astype(int)
    y = LabelEncoder().fit_transform(y)
    X = MinMaxScaler().fit_transform(X)
    return X, y, drivers, mask

class MLP(nn.Module):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim,64), nn.ReLU(),
                                 nn.Linear(64,64), nn.ReLU(),
                                 nn.Linear(64,out_dim))
    def forward(self,x): return self.net(x)

class RDataset(Dataset):
    def __init__(self,X,y): self.X=torch.tensor(X,dtype=torch.float32); self.y=torch.tensor(y,dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self,i): return self.X[i], self.y[i]

def train_once(X,y):
    in_dim=X.shape[1]; out_dim=len(np.unique(y))
    cw=torch.tensor(compute_class_weight("balanced",classes=np.unique(y),y=y),dtype=torch.float32)
    model=MLP(in_dim,out_dim); opt=torch.optim.Adam(model.parameters(),lr=0.001)
    loss_fn=nn.CrossEntropyLoss(weight=cw); loader=DataLoader(RDataset(X,y),batch_size=512,shuffle=True)
    for epoch in range(20):
        for bx,by in loader:
            logits=model(bx); loss=loss_fn(logits,by)
            opt.zero_grad(); loss.backward(); opt.step()
    return model

def simulate(model, drivers, ref_path, out_path, lock_class=0):
    full_stack=np.stack(drivers,axis=-1); flat=full_stack.reshape(-1,full_stack.shape[-1])
    valid_mask = np.all(~np.isnan(flat),axis=-1); flat_valid = (flat[valid_mask]-flat[valid_mask].min(0))/(flat[valid_mask].ptp(0)+1e-9)
    with torch.no_grad():
        logits=model(torch.tensor(flat_valid,dtype=torch.float32))
        preds = torch.softmax(logits,dim=1).argmax(1).numpy()
    corrected=np.full(flat.shape[0],np.nan); corrected[valid_mask]=preds
    ref_map,_ = read_r(landuse_2010); ref_flat=ref_map.flatten(); corrected[ref_flat==lock_class]=lock_class
    out_arr=corrected.reshape(full_stack.shape[:-1])
    with rasterio.open(ref_path) as ref:
        profile=ref.profile; profile.update(dtype=rasterio.uint8,count=1)
        with rasterio.open(out_path,"w",**profile) as dst: dst.write(out_arr.astype(np.uint8),1)
    return out_arr

# Train on 2010→2017, simulate 2030
X1,y1,drivers1,_ = prepare_X_y(landuse_2010, landuse_2017)
model_2030 = train_once(X1,y1)
ref = rasterio.open(landuse_2010)
sim2030 = simulate(model_2030, drivers1, landuse_2010, f"{rroot}/simulated_landuse_2030.tif")

# Train on 2017→sim2030, simulate 2050
X2,y2,drivers2,_ = prepare_X_y(landuse_2017, f"{rroot}/simulated_landuse_2030.tif")
model_2050 = train_once(X2,y2)
sim2050 = simulate(model_2050, drivers2, landuse_2010, f"{rroot}/simulated_landuse_2050.tif")

plt.imshow(sim2050, cmap="tab20"); plt.title("Unconstrained Scenario — 2050"); plt.axis("off"); plt.show()

# ============================================================
# F) Landscape-method scenario (growth constraints; class-0 lock)
#     Training pairs also updated to 2010→2017 then 2017→2030
# ============================================================
def simulate_future(model, drivers, ref_path, base_map, out_path, base_ref):
    full_stack=np.stack(drivers,axis=-1); flat=full_stack.reshape(-1,full_stack.shape[-1])
    valid_mask=np.all(~np.isnan(flat),axis=-1); flat_valid=(flat[valid_mask]-flat[valid_mask].min(0))/(flat[valid_mask].ptp(0)+1e-9)
    with torch.no_grad():
        logits=model(torch.tensor(flat_valid,dtype=torch.float32))
        preds=torch.softmax(logits,dim=1).argmax(1).numpy()
    corrected=np.full(flat.shape[0],np.nan); corrected[valid_mask]=preds
    base_flat=base_map.flatten(); ref_flat=base_ref.flatten(); corrected[ref_flat==0]=0
    for cls in range(1,10):
        ref_count=np.sum(ref_flat==cls); pred_idx=np.where(corrected==cls)[0]
        if cls in [1,2]:   min_n=ref_count+1;      max_n=ref_count*1.10
        elif cls in [4,5]: min_n=ref_count+1;      max_n=ref_count*1.05
        elif cls==9:       min_n=ref_count*0.80;   max_n=ref_count*0.90
        else:              min_n=ref_count*0.95;   max_n=ref_count*1.05
        cur=len(pred_idx)
        if cur>max_n:
            drop=int(cur-max_n); corrected[pred_idx[:drop]]=base_flat[pred_idx[:drop]]
        elif cur<min_n:
            alt=np.where((corrected!=cls)&(ref_flat!=0))[0]; fill=int(min_n-cur); corrected[alt[:fill]]=cls
    out_arr=corrected.reshape(full_stack.shape[:-1])
    with rasterio.open(ref_path) as ref:
        profile=ref.profile; profile.update(dtype=rasterio.uint8,count=1)
        with rasterio.open(out_path,"w",**profile) as dst: dst.write(out_arr.astype(np.uint8),1)
    return out_arr

# 2010→2017 train → simulate 2030 (constrained)
X1_l,y1_l,drivers1_l,_ = prepare_X_y(landuse_2010, landuse_2017)
model_2030_l = train_once(X1_l,y1_l)
base_map_2017,_ = read_r(landuse_2017); base_ref,_ = read_r(landuse_2010)
sim2030_l = simulate_future(model_2030_l, drivers1_l, landuse_2010, base_map_2017, f"{rroot}/simulated_landuse_2030.tif", base_ref)

# 2017→2030 train → simulate 2050 (constrained)
X2_l,y2_l,drivers2_l,_ = prepare_X_y(landuse_2017, f"{rroot}/simulated_landuse_2030.tif")
model_2050_l = train_once(X2_l,y2_l)
base_map_2030,_ = read_r(f"{rroot}/simulated_landuse_2030.tif")
sim2050_l = simulate_future(model_2050_l, drivers2_l, landuse_2010, base_map_2030, f"{rroot}/simulated_landuse_2050.tif", base_ref)

plt.imshow(sim2050_l, cmap="tab20"); plt.title("Landscape-Method Scenario — 2050"); plt.axis("off"); plt.show()

# ============================================================
# G) BAU baseline via transition matrices
#     Updated to 2010→2017 matrix; forecast 2030 & 2050
# ============================================================
rroot_bau = "/content/"  # <- note: this baseline reads from /content/
landuse_2010_b = f"{rroot_bau}/2010_9.tif"
landuse_2017_b = f"{rroot_bau}/2017_9.tif"
study_shp_b = "study_area.shp"

def read_b(p):
    with rasterio.open(p) as src: return src.read(1), src.profile
def load_mask_b(shp, ref_r):
    gdf=gpd.read_file(shp);
    with rasterio.open(ref_r) as src: out,_=rasterio.mask.mask(src,gdf.geometry,crop=False);
    return out[0].astype(bool)
def save_b(arr,path,profile):
    profile.update(dtype=rasterio.uint8,count=1)
    with rasterio.open(path,"w",**profile) as dst: dst.write(arr.astype(np.uint8),1)
def trans_matrix(a,b,mask):
    a=a[:mask.shape[0],:mask.shape[1]].astype(int); b=b[:mask.shape[0],:mask.shape[1]].astype(int)
    valid = mask & (~np.isnan(a)) & (~np.isnan(b))
    M=np.zeros((10,10))
    for i in range(10):
        from_idx=(a==i)&valid; total=np.sum(from_idx)
        if total==0: continue
        for j in range(10): M[i,j]=np.sum((b==j)&from_idx)/total
    return M
def simulate_by_M(cur, M, mask, seed=42):
    np.random.seed(seed)
    cur=cur[:mask.shape[0],:mask.shape[1]].astype(int); nxt=cur.copy()
    R,C=cur.shape
    for r in tqdm(range(R)):
        for c in range(C):
            if not mask[r,c]: continue
            cl=int(cur[r,c])
            if cl==0: nxt[r,c]=0
            elif 0<=cl<10: nxt[r,c]=np.random.choice(np.arange(10), p=M[cl])
            else: nxt[r,c]=cl
    return nxt

ref_map, profile_b = read_b(landuse_2010_b)
land2017,_ = read_b(landuse_2017_b)
mask_b = load_mask_b(study_shp_b, landuse_2010_b)
M_10_17 = trans_matrix(ref_map, land2017, mask_b)
print("✅ Transition matrix 2010→2017 computed")

sim2030_b = simulate_by_M(land2017, M_10_17, mask_b, seed=123)
save_b(sim2030_b, f"{rroot_bau}/simulated_landuse_2030.tif", profile_b)

M_17_30 = trans_matrix(land2017, sim2030_b, mask_b)
sim2050_b = simulate_by_M(sim2030_b, M_17_30, mask_b, seed=456)
save_b(sim2050_b, f"{rroot_bau}/simulated_landuse_2050.tif", profile_b)

print("✅ BAU 2030/2050 exported.")
